In [3]:
#! /usr/bin/env python
"""
Compute debris thickness through sub-debris and temperature inversion methods
"""

import sys
import os
import re
import subprocess
from datetime import datetime, timedelta
import time
import pickle
from collections import OrderedDict

import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rasterio
import xarray as xr
from osgeo import gdal, ogr, osr

from pygeotools.lib import malib, warplib, geolib, iolib, timelib

import debrisglobal.globaldebris_input as debris_prms
from debrisglobal.glacfeat import GlacFeat, create_glacfeat

debug=False

In [4]:
# Mosaic multiple files
mosaic_ts_info = False
from rasterio.merge import merge

# ====== MERGE CALIBRATED GLACIERS INTO DEGREE PIXELS ======
# Export all .tifs to a common projection that will enable merging them smoothly
dst_crs = 'EPSG:4326'

if mosaic_ts_info:
    
    # All Ts filenames
    ts_fullfns = []
    ts_year_fullfns = []
    ts_doy_fullfns = []
    ts_dayfrac_fullfns = []
    for i in os.listdir(debris_prms.ts_fp):
        if i.endswith('.tif'):
            if 'tsurfC' in i:
                ts_fullfns.append(debris_prms.ts_fp + i)
            elif 'year' in i:
                ts_year_fullfns.append(debris_prms.ts_fp + i)
            elif 'doy' in i:
                ts_doy_fullfns.append(debris_prms.ts_fp + i)
            elif 'dayfrac' in i:
                ts_dayfrac_fullfns.append(debris_prms.ts_fp + i)

    ts_fullfns = sorted(ts_fullfns)
    ts_year_fullfns = sorted(ts_year_fullfns)
    ts_doy_fullfns = sorted(ts_doy_fullfns)
    ts_dayfrac_fullfns = sorted(ts_dayfrac_fullfns)
    
    if debug:
        print(ts_fullfns)
        print(ts_year_fullfns)
        print(ts_doy_fullfns)
        print(ts_dayfrac_fullfns)
        
    

    def mosaic_fullfns(list_fullfns, fn_merged=None):
        if fn_merged is None:
            # Hack to deal with naming convention I used of '16-03_...' in GEE
            if debris_prms.roi not in ['10', '16']:
                fn_merged = list_fullfns[0].split('/')[-1].split('-')[0] + '.tif'
            else:
                fn_merged = (list_fullfns[0].split('/')[-1].split('-')[0] + '-' + 
                             list_fullfns[0].split('/')[-1].split('-')[1] + '.tif')
        src_files_to_mosaic = []
        for fn in list_fullfns:
            with rasterio.open(fn) as src:
                src = rasterio.open(fn)
                src_files_to_mosaic.append(src)

        mos, out_trans = merge(src_files_to_mosaic)

        with rasterio.open(debris_prms.ts_fp + fn_merged,"w", driver ='Gtiff',count=1,
                           height= mos.shape[1],
                           width= mos.shape[2],
                           transform= out_trans,
                           crs= src.crs,
                           dtype= src.dtypes[0]) as dest:
            dest.write(mos)
            
    if len(ts_fullfns) > 1:
        mosaic_fullfns(ts_fullfns)
    if len(ts_year_fullfns) > 1:
        mosaic_fullfns(ts_year_fullfns)
    if len(ts_doy_fullfns) > 1:
        mosaic_fullfns(ts_doy_fullfns)
    if len(ts_dayfrac_fullfns) > 1:
        mosaic_fullfns(ts_dayfrac_fullfns)

In [5]:
ts_fullfns = debris_prms.ts_fullfns_dict[debris_prms.roi]
ts_year_fullfns = debris_prms.ts_year_fullfns_dict[debris_prms.roi]
ts_doy_fullfns = debris_prms.ts_doy_fullfns_dict[debris_prms.roi]
ts_dayfrac_fullfns = debris_prms.ts_dayfrac_fullfns_dict[debris_prms.roi]

In [ ]:
add_ts_fns = True
if add_ts_fns:
    
    # Load all debris-covered glaciers
    dc_shp = gpd.read_file(debris_prms.debriscover_fp + debris_prms.debriscover_fn_dict[debris_prms.roi])
    dc_shp = dc_shp.sort_values(by=['RGIId'])
    dc_shp.reset_index(inplace=True, drop=True)
    
    dc_rgiids = [x.split('-')[1] for x in dc_shp.RGIId.values]
    print(len(dc_rgiids), 'ex:', dc_rgiids[0],'\n')
    main_glac_rgi_all = debris_prms.selectglaciersrgitable(dc_rgiids)

    # Ts filenames dataframe
    if os.path.exists(debris_prms.ts_fp + debris_prms.ts_fns_fn):
        ts_fns_df = pd.read_csv(debris_prms.ts_fp + debris_prms.ts_fns_fn)
    else:
        ts_fns_cns = ['RGIId', 'ts_fullfn', 'ts_year_fullfn', 'ts_doy_fullfn', 'ts_dayfrac_fullfn']
        ts_fns_df = pd.DataFrame(np.zeros((main_glac_rgi_all.shape[0], len(ts_fns_cns))), columns=ts_fns_cns)
        ts_fns_df['RGIId'] = main_glac_rgi_all['RGIId']
        
    if debug:
        print(ts_fullfns)
        print(ts_year_fullfns)
        print(ts_doy_fullfns)
        print(ts_dayfrac_fullfns)
        
    ts_fn_wglacier = None
    ts_year_fn_wglacier = None
    ts_doy_fn_wglacier = None
    ts_dayfrac_fn_wglacier = None
    
    # ===== PROCESS EACH GLACIER =====
    for nglac, glac_idx in enumerate(main_glac_rgi_all.index.values):
#     for nglac, glac_idx in enumerate([main_glac_rgi_all.index.values[0]]):
        glac_str = main_glac_rgi_all.loc[glac_idx,'rgino_str']
        rgiid = main_glac_rgi_all.loc[glac_idx,'RGIId']
        region = glac_str.split('.')[0]

        if int(region) < 10:
            glac_str_noleadzero = str(int(glac_str.split('.')[0])) + '.' + glac_str.split('.')[1]
        else:
            glac_str_noleadzero = glac_str

        print(nglac, glac_idx, rgiid)

        # Create glacier feature from ice thickness raster
        thick_dir = debris_prms.oggm_fp + 'thickness/RGI60-' + str(region.zfill(2)) + '/'
        thick_fn = 'RGI60-' + str(region.zfill(2)) + '.' + rgiid.split('.')[1] + '_thickness.tif'

        if os.path.exists(thick_dir + thick_fn):
#         if os.path.exists(thick_dir + thick_fn) and '0.0' in ts_fns_df.loc[glac_idx,:].values:
#             print(nglac, glac_idx, rgiid)

            gf = create_glacfeat(thick_dir, thick_fn)

            # Debris shape layer processing
            dc_shp_proj_fn = (debris_prms.glac_shp_proj_fp + glac_str + '_dc_crs' + 
                              str(gf.aea_srs.GetAttrValue("AUTHORITY", 1)) + '.shp')
            if os.path.exists(dc_shp_proj_fn) == False:
                dc_shp_init = gpd.read_file(debris_prms.debriscover_fp + debris_prms.debriscover_fn_dict[debris_prms.roi])
                dc_shp_single = dc_shp_init[dc_shp_init['RGIId'] == rgiid]
                dc_shp_single = dc_shp_single.reset_index()
                dc_shp_proj = dc_shp_single.to_crs({'init': 'epsg:' + str(gf.aea_srs.GetAttrValue("AUTHORITY", 1))})
                dc_shp_proj.to_file(dc_shp_proj_fn)
            dc_shp_ds = ogr.Open(dc_shp_proj_fn, 0)
            dc_shp_lyr = dc_shp_ds.GetLayer()

            # ==== CHECK IF TIF HAS SURFACE TEMPERATURE DATA OVER THE GLACIER =====
            # Add the filenames
            fn_dict = OrderedDict()
            # DEM
            z1_fp = debris_prms.oggm_fp + 'dems/RGI60-' + str(region.zfill(2)) + '/'
            z1_fn = 'RGI60-' + str(region.zfill(2)) + '.' + rgiid.split('.')[1] + '_dem.tif'
            fn_dict['z1'] = z1_fp + z1_fn
            # Ice thickness
            thick_dir = debris_prms.oggm_fp + 'thickness/RGI60-' + str(region.zfill(2)) + '/'
            thick_fn = 'RGI60-' + str(region.zfill(2)) + '.' + rgiid.split('.')[1] + '_thickness.tif'
            fn_dict['ice_thick'] = thick_dir + thick_fn
            # Ts
            ts_dict_keys = []
            for nfn, ts_fn in enumerate(ts_fullfns):
                fn_dict['ts_' + str(nfn)] = ts_fn
                ts_dict_keys.append('ts_' + str(nfn))
            ts_year_dict_keys = []
            for nfn, ts_year_fn in enumerate(ts_year_fullfns):
                fn_dict['ts_year_' + str(nfn)] = ts_year_fn
                ts_year_dict_keys.append('ts_year_' + str(nfn))
            ts_doy_dict_keys = []
            for nfn, ts_doy_fn in enumerate(ts_doy_fullfns):
                fn_dict['ts_doy_' + str(nfn)] = ts_doy_fn
                ts_doy_dict_keys.append('ts_doy_' + str(nfn))
            ts_dayfrac_dict_keys = []  
            for nfn, ts_dayfrac_fn in enumerate(ts_dayfrac_fullfns):
                fn_dict['ts_dayfrac_' + str(nfn)] = ts_dayfrac_fn
                ts_dayfrac_dict_keys.append('ts_dayfrac_' + str(nfn))

            # ===== PROCESS THE DATA =====
            #Expand extent to include buffered region around glacier polygon
            warp_extent = geolib.pad_extent(gf.glac_geom_extent, width=debris_prms.buff_dist)
            #Warp everything to common res/extent/proj
            z1_gt = gdal.Open(fn_dict['z1']).GetGeoTransform()
            z1_res = np.min([z1_gt[1], -z1_gt[5]])
            ds_list = warplib.memwarp_multi_fn(fn_dict.values(), res=z1_res, extent=warp_extent, 
                                               t_srs=gf.aea_srs, verbose=False, r='cubic')
            ds_dict = dict(zip(fn_dict.keys(), ds_list))
            gf.ds_dict = ds_dict

            if 'z1' in ds_dict:
                #This is False over glacier polygon surface, True elsewhere - can be applied directly
                glac_geom_mask = geolib.geom2mask(gf.glac_geom, ds_dict['z1'])
                glac_geom_mask_copy = glac_geom_mask.copy()
                gf.z1 = np.ma.array(iolib.ds_getma(ds_dict['z1']), mask=glac_geom_mask)
                # Debris cover
                dc_shp_lyr_mask = geolib.lyr2mask(dc_shp_lyr, ds_dict['ice_thick'])
                gf.dc_mask = np.ma.mask_or(dc_shp_lyr_mask, glac_geom_mask)
                gf.dc_area = np.ma.array(iolib.ds_getma(ds_dict['z1']), mask=gf.dc_mask)

                # ====== SURFACE TEMPERATURE =====
                find_ts = True
                ts_key_count = 0
                if debug:
                    print('\nSURFACE TEMPERATURE')
                while find_ts and ts_key_count < len(ts_dict_keys):
                    ts_key = ts_dict_keys[ts_key_count]
                    if debug:
                        print(ts_key, ts_fullfns[ts_key_count].split('/')[-1])
                    gf.ts = np.ma.array(iolib.ds_getma(ds_dict[ts_key]), mask=glac_geom_mask_copy)
                    gf.ts.mask = np.ma.mask_or(
                        glac_geom_mask, np.ma.getmask(np.ma.masked_array(gf.ts.data, np.isnan(gf.ts.data))))
                    gf.dc_ts = np.ma.array(iolib.ds_getma(ds_dict[ts_key]), mask=glac_geom_mask_copy)
                    gf.dc_ts.mask = gf.dc_mask
                    # Count pixels
                    dc_ts_pixels = len(gf.ts.nonzero()[0])
                    if dc_ts_pixels / gf.dc_area.count() * 100 > 75:
                        find_ts = False
                        # Export raster
                        ts_fn = rgiid + '_ts.tif'
                        ts_fp = debris_prms.oggm_ts_fp + 'RGI60-' + debris_prms.roi + '/'
                        if not os.path.exists(ts_fp):
                            os.makedirs(ts_fp)
                        iolib.writeGTiff(gf.ts, ts_fp + ts_fn, gf.ds_dict['z1'])
                        ts_fn_wglacier = ts_fp + ts_fn
                        
                        if debug:
                            print('\n# z1 dc pixels:', gf.dc_area.count())
                            print('# dc_ts_pixels:', dc_ts_pixels)
                            print('ts_fn:', ts_fn_wglacier)
                            var_full2plot = gf.ts.copy()
                            clim = malib.calcperc(var_full2plot, (2,98))
                            plot_array(var_full2plot, clim, [glac_str + ' Ts'], 'inferno', 'Ts (degC)', 
                                       close_fig=False)
                    ts_key_count += 1
                    
                # ====== TS YEAR =====
                find_ts_year = True
                ts_key_count = 0
                if debug:
                    print('\nYEAR')
                while find_ts_year and ts_key_count < len(ts_year_dict_keys):
                    ts_key = ts_year_dict_keys[ts_key_count]
                    if debug:
                        print(ts_key, ts_year_fullfns[ts_key_count].split('/')[-1])
                    gf.ts = np.ma.array(iolib.ds_getma(ds_dict[ts_key]), mask=glac_geom_mask_copy)
                    gf.ts.mask = np.ma.mask_or(
                        glac_geom_mask, np.ma.getmask(np.ma.masked_array(gf.ts.data, np.isnan(gf.ts.data))))
                    gf.dc_ts = np.ma.array(iolib.ds_getma(ds_dict[ts_key]), mask=glac_geom_mask_copy)
                    gf.dc_ts.mask = gf.dc_mask
                    # Count Ts pixels
                    dc_ts_pixels = len(gf.ts.nonzero()[0])
                    if dc_ts_pixels / gf.dc_area.count() * 100 > 75:
                        find_ts_year = False
                        # Export raster
                        ts_year_fn = rgiid + '_ts_year.tif'
                        ts_year_fp = debris_prms.oggm_ts_info_fp + 'RGI60-' + debris_prms.roi + '/'
                        if not os.path.exists(ts_year_fp):
                            os.makedirs(ts_year_fp)
                        iolib.writeGTiff(gf.ts, ts_year_fp + ts_year_fn, gf.ds_dict['z1'])
                        ts_year_fn_wglacier = ts_year_fp + ts_year_fn
                        
                        if debug:
                            print('\n# z1 dc pixels:', gf.dc_area.count())
                            print('# dc_ts_pixels:', dc_ts_pixels)
                            print('ts_year_fn:', ts_year_fn_wglacier)
                            var_full2plot = gf.ts.copy()
                            clim = (2013,2018)
                            plot_array(var_full2plot, clim, [glac_str + ' Ts_year'], 'inferno', 'Year', 
                                       close_fig=False)
                    ts_key_count += 1
                    
                # ====== TS DOY =====
                find_ts_doy = True
                ts_key_count = 0
                if debug:
                    print('\nDAY OF YEAR')
                while find_ts_doy and ts_key_count < len(ts_doy_dict_keys):
                    ts_key = ts_doy_dict_keys[ts_key_count]
                    if debug:
                        print(ts_key, ts_doy_fullfns[ts_key_count].split('/')[-1])
                    gf.ts = np.ma.array(iolib.ds_getma(ds_dict[ts_key]), mask=glac_geom_mask_copy)
                    gf.ts.mask = np.ma.mask_or(
                        glac_geom_mask, np.ma.getmask(np.ma.masked_array(gf.ts.data, np.isnan(gf.ts.data))))
                    gf.dc_ts = np.ma.array(iolib.ds_getma(ds_dict[ts_key]), mask=glac_geom_mask_copy)
                    gf.dc_ts.mask = gf.dc_mask
                    # Count Ts pixels
                    dc_ts_pixels = len(gf.ts.nonzero()[0])
                    if dc_ts_pixels / gf.dc_area.count() * 100 > 75:
                        find_ts_doy = False
                        # Export raster
                        ts_doy_fn = rgiid + '_ts_doy.tif'
                        ts_doy_fp = debris_prms.oggm_ts_info_fp + 'RGI60-' + debris_prms.roi + '/'
                        if not os.path.exists(ts_doy_fp):
                            os.makedirs(ts_doy_fp)
                        iolib.writeGTiff(gf.ts, ts_doy_fp + ts_doy_fn, gf.ds_dict['z1'])
                        ts_doy_fn_wglacier = ts_doy_fp + ts_doy_fn
                        if debug:
                            print('\n# z1 dc pixels:', gf.dc_area.count())
                            print('# dc_ts_pixels:', dc_ts_pixels)
                            print('ts_doy_fn:', ts_doy_fn_wglacier)
                            var_full2plot = gf.ts.copy()
                            clim = (160,280)
                            plot_array(var_full2plot, clim, [glac_str + ' Ts_doy'], 'inferno', 'DOY', 
                                       close_fig=False)
                    ts_key_count += 1
                    
                # ====== TS DAYFRAC =====
                find_ts_dayfrac = True
                ts_key_count = 0
                if debug:
                    print('\nDAY FRACTION')
                while find_ts_dayfrac and ts_key_count < len(ts_dayfrac_dict_keys):
                    ts_key = ts_dayfrac_dict_keys[ts_key_count]
                    if debug:
                        print(ts_key, ts_dayfrac_fullfns[ts_key_count].split('/')[-1])
                    gf.ts = np.ma.array(iolib.ds_getma(ds_dict[ts_key]), mask=glac_geom_mask_copy)
                    gf.ts.mask = np.ma.mask_or(
                        glac_geom_mask, np.ma.getmask(np.ma.masked_array(gf.ts.data, np.isnan(gf.ts.data))))
                    gf.dc_ts = np.ma.array(iolib.ds_getma(ds_dict[ts_key]), mask=glac_geom_mask_copy)
                    gf.dc_ts.mask = gf.dc_mask
                    # Count Ts pixels
                    dc_ts_pixels = len(gf.ts.nonzero()[0])
                    if dc_ts_pixels / gf.dc_area.count() * 100 > 75:
                        find_ts_dayfrac = False
                        # Export raster
                        ts_dayfrac_fn = rgiid + '_ts_dayfrac.tif'
                        ts_dayfrac_fp = debris_prms.oggm_ts_info_fp + 'RGI60-' + debris_prms.roi + '/'
                        if not os.path.exists(ts_dayfrac_fp):
                            os.makedirs(ts_dayfrac_fp)
                        iolib.writeGTiff(gf.ts, ts_dayfrac_fp + ts_dayfrac_fn, gf.ds_dict['z1'])
                        ts_dayfrac_fn_wglacier = ts_dayfrac_fp + ts_dayfrac_fn
                        if debug:
                            print('\n# z1 dc pixels:', gf.dc_area.count())
                            print('# dc_ts_pixels:', dc_ts_pixels)
                            print('ts_dayfrac_fn:', ts_dayfrac_fn_wglacier)
                            var_full2plot = gf.ts.copy()
                            clim = (0,24)
                            plot_array(var_full2plot, clim, [glac_str + ' Ts_dayfrac'], 'inferno', 'dayfrac', 
                                       close_fig=False)
                    ts_key_count += 1
                    
                
                        
            # ===== Add layers =====
            # Record filenames if all exist
            if (ts_fn_wglacier is not None and ts_year_fn_wglacier is not None and ts_doy_fn_wglacier is not None and 
                ts_dayfrac_fn_wglacier is not None):                    
                ts_fns_df.loc[glac_idx,:] = [rgiid, ts_fn_wglacier, ts_year_fn_wglacier, ts_doy_fn_wglacier, 
                                             ts_dayfrac_fn_wglacier]
                    
    # Save updated filenames
    ts_fns_df.to_csv(debris_prms.ts_fp + debris_prms.ts_fns_fn, index=False)

6074 ex: 17.00001 

6074 glaciers in region 17 are included in this model run: ['00001', '00008', '00009', '00011', '00013', '00014', '00015', '00016', '00021', '00022', '00023', '00024', '00029', '00031', '00032', '00055', '00064', '00065', '00066', '00067', '00068', '00070', '00074', '00080', '00081', '00083', '00095', '00102', '00122', '00134', '00136', '00141', '00144', '00148', '00172', '00271', '00312', '00367', '00368', '00396', '00412', '00421', '00424', '00426', '00456', '00466', '00467', '00468', '00469', '00471'] and more
This study is focusing on 6074 glaciers in region [17]
0 0 RGI60-17.00001
1 1 RGI60-17.00008
2 2 RGI60-17.00009
3 3 RGI60-17.00011
4 4 RGI60-17.00013
5 5 RGI60-17.00014
6 6 RGI60-17.00015
7 7 RGI60-17.00016
8 8 RGI60-17.00021
9 9 RGI60-17.00022


/Users/davidrounce/anaconda3/envs/debris_thickness_global/lib/python3.6/site-packages/pyproj/crs.py:77: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method.
  return _prepare_from_string(" ".join(pjargs))


10 10 RGI60-17.00023
11 11 RGI60-17.00024
12 12 RGI60-17.00029
13 13 RGI60-17.00031
14 14 RGI60-17.00032
15 15 RGI60-17.00055
16 16 RGI60-17.00064
17 17 RGI60-17.00065
18 18 RGI60-17.00066
19 19 RGI60-17.00067
20 20 RGI60-17.00068
21 21 RGI60-17.00070
22 22 RGI60-17.00074
23 23 RGI60-17.00080
24 24 RGI60-17.00081
25 25 RGI60-17.00083
26 26 RGI60-17.00095
27 27 RGI60-17.00102
28 28 RGI60-17.00122
29 29 RGI60-17.00134
30 30 RGI60-17.00136
31 31 RGI60-17.00141
32 32 RGI60-17.00144
33 33 RGI60-17.00148
34 34 RGI60-17.00172
35 35 RGI60-17.00271
36 36 RGI60-17.00312
37 37 RGI60-17.00367
38 38 RGI60-17.00368
39 39 RGI60-17.00396
40 40 RGI60-17.00412
41 41 RGI60-17.00421
42 42 RGI60-17.00424
43 43 RGI60-17.00426
44 44 RGI60-17.00456
45 45 RGI60-17.00466
46 46 RGI60-17.00467
47 47 RGI60-17.00468
48 48 RGI60-17.00469
49 49 RGI60-17.00471
50 50 RGI60-17.00478
51 51 RGI60-17.00480
52 52 RGI60-17.00484
53 53 RGI60-17.00492
54 54 RGI60-17.00496
55 55 RGI60-17.00500
56 56 RGI60-17.00501
57 57 RGI60-1

In [ ]:
print('\nDONE\n')